Merge multiple videos and export a GIF or MP4 optimized for GitHub / social media.
Zoom-in on specific moments is available as an optional feature for screen-recording demos.
Supported input formats: `.mov` `.mp4` `.avi` `.mkv` `.webm`


##### Choosing an output format

| Format | Characteristics | Best for | Not ideal for |
|---|---|---|---|
| GIF | Auto-play, loops, works in any Markdown renderer. 256 colors, larger file size | GitHub README (auto-play required) · Zenn / Qiita | Size-constrained scenarios |
| MP4 | High quality, small file, full color. Requires a player. **※ No audio** | X / Slack / GitHub README (click-to-play is fine) | Scenarios requiring auto-play or looping |


##### How to use — just run cells top to bottom

| Step | Description | Action |
|:---:|---|---|
| ① | Setup | Automatic |
| ② | Upload videos | File selection dialog |
| ③ | Set merge order | Dropdown (skipped for single file) |
| ④ | Configure format, quality & speed | Radio buttons |
| ⑤ | Generate preview | Automatic |
| ⑥ | Review preview (play MP4, check timestamps) | Automatic |
| ⑦ | Zoom settings (optional) | Checkbox + event input |
| ⑧ | Final export | Automatic |
| ⑨ | Choose destination and save | Radio buttons |

> ※ Upload limit (②): ~2 GB per file; multiple files scale as `2 GB × number of files`


##### Reference: platform size limits (as of 2026/02)

| Platform | Formats | Size limit |
|---|---|---|
| GitHub README | GIF / MP4 / MOV | 100 MB (video) / 10 MB (GIF) |
| Zenn | GIF | 3 MB |
| Qiita | GIF / MP4 | 100 MB |
| X (standard) | MP4 / MOV | 512 MB |
| Slack | GIF / MP4 / MOV etc. | 1 GB |

In [ ]:
# ① Setup
!apt-get install -y ffmpeg -q

from google.colab import files as colab_files, drive
import os, shutil, subprocess, ipywidgets as w
from IPython.display import display, Image, Video

def _ffmpeg(*args):
    """Run ffmpeg with the given argument list. Uses subprocess to avoid shell quoting issues."""
    cmd = ['ffmpeg', '-y', '-loglevel', 'warning'] + [str(a) for a in args]
    proc = subprocess.run(cmd, capture_output=True, text=True)
    if proc.returncode != 0:
        print('❌ ffmpeg error:\n' + proc.stderr[-2000:])
        raise RuntimeError(f'ffmpeg failed (returncode={proc.returncode})')
    return proc.returncode

_ZOOM_INTERMEDIATE_CRF = 18  # Intermediate MP4 for zoom; does not affect final GIF quality

print('✅ Ready')

In [ ]:
# ② Upload (multiple files allowed)
print('📂 Select video files (supported: .mov .mp4 .avi .mkv .webm)')
uploaded = colab_files.upload()

VIDEO_EXT = ('.mov', '.mp4', '.avi', '.mkv', '.webm')
uploaded_videos = [f for f in uploaded.keys() if f.lower().endswith(VIDEO_EXT)]

if not uploaded_videos:
    print('❌ No video files found')
else:
    print(f'\n✅ {len(uploaded_videos)} file(s) detected:')
    for i, f in enumerate(uploaded_videos):
        size = os.path.getsize(f) / 1024 / 1024
        print(f'   [{i}] {f} ({size:.1f} MB)')

In [ ]:
# ③ Merge order
if len(uploaded_videos) == 1:
    order_widgets = None
    print(f'✅ Single file — skipping order selection: {uploaded_videos[0]}')
else:
    print('🔢 Set the merge order')
    order_widgets = [
        w.Dropdown(options=uploaded_videos, value=uploaded_videos[i],
                   description=f'#{i+1}:', style={'description_width': '60px'},
                   layout=w.Layout(width='420px'))
        for i in range(len(uploaded_videos))
    ]
    display(w.VBox(order_widgets))

In [ ]:
# ④ Output format, quality & speed

GIF_PRESETS = {
    'github': {'scale': 960,  'fps': 15, 'colors': 256, 'dither': 'floyd_steinberg'},
    'sns':    {'scale': 640,  'fps': 10, 'colors': 128, 'dither': 'bayer'},
    'hq':     {'scale': 1280, 'fps': 20, 'colors': 256, 'dither': 'floyd_steinberg'},
}

def make_toggle(widget, show_when):
    def _cb(change):
        widget.layout.display = '' if change['new'] == show_when else 'none'
    return _cb

format_sel = w.RadioButtons(
    options=[
        ('GIF — looping · GitHub README', 'gif'),
        ('MP4 (H.264) — high quality · SNS / Slack', 'mp4'),
    ],
    value='gif', description='Format:', style={'description_width': '60px'},
    layout=w.Layout(width='440px')
)

# GIF settings
gif_quality_sel = w.RadioButtons(
    options=[
        ('GitHub README  960px / 15fps',  'github'),
        ('Lightweight    640px / 10fps',  'sns'),
        ('High quality  1280px / 20fps',  'hq'),
        ('Custom',                        'custom'),
    ],
    value='github', description='Quality:', style={'description_width': '60px'},
    layout=w.Layout(width='320px')
)
gif_custom_scale  = w.BoundedIntText(value=960,  min=240, max=1920, description='Width px:',       layout=w.Layout(width='220px'), style={'description_width': '70px'})
gif_custom_fps    = w.BoundedIntText(value=15,   min=1,   max=30,   description='FPS:',             layout=w.Layout(width='180px'), style={'description_width': '70px'})
gif_custom_colors = w.BoundedIntText(value=256,  min=16,  max=256,  description='Colors (16-256):', layout=w.Layout(width='220px'), style={'description_width': '100px'})
gif_custom_dither = w.Dropdown(
    options=[('bayer (lightweight)', 'bayer'), ('floyd_steinberg (high quality)', 'floyd_steinberg')],
    value='floyd_steinberg',
    description='Dithering:', layout=w.Layout(width='320px'), style={'description_width': '80px'}
)
gif_custom_box = w.VBox(
    [gif_custom_scale, gif_custom_fps, gif_custom_colors, gif_custom_dither],
    layout=w.Layout(margin='0 0 0 20px', display='none')
)
gif_box = w.VBox([w.HTML('<b>🎨 GIF Quality</b>'), gif_quality_sel, gif_custom_box])
gif_quality_sel.observe(make_toggle(gif_custom_box, 'custom'), names='value')

# MP4 settings
mp4_quality_sel = w.RadioButtons(
    options=[
        ('High quality  (CRF 18) — larger file',  18),
        ('Standard      (CRF 23) — balanced',     23),
        ('Lightweight   (CRF 28) — smaller file', 28),
        ('Custom',                                -1),
    ],
    value=23, description='Quality:', style={'description_width': '60px'},
    layout=w.Layout(width='320px')
)
mp4_custom_crf = w.BoundedIntText(
    value=23, min=0, max=51, description='CRF (0-51):',
    layout=w.Layout(width='220px', display='none'), style={'description_width': '90px'}
)
mp4_scale = w.BoundedIntText(value=960, min=240, max=1920, description='Width px:', layout=w.Layout(width='220px'), style={'description_width': '70px'})
mp4_fps   = w.BoundedIntText(value=30,  min=1,   max=60,   description='FPS:',      layout=w.Layout(width='180px'), style={'description_width': '70px'})
mp4_box = w.VBox([
    w.HTML('<b>🎨 MP4 Quality</b>'), mp4_quality_sel, mp4_custom_crf,
    w.HTML('<b>📐 Resolution & Frame rate</b>'), mp4_scale, mp4_fps,
])
mp4_quality_sel.observe(make_toggle(mp4_custom_crf, -1), names='value')
settings_container = w.VBox([gif_box])

def on_format_change(change):
    settings_container.children = [gif_box] if change['new'] == 'gif' else [mp4_box]
format_sel.observe(on_format_change, names='value')

# Playback speed
speed_sel = w.RadioButtons(
    options=[('1.0x (original)', 1.0), ('1.5x', 1.5), ('2.0x', 2.0), ('3.0x', 3.0), ('0.75x', 0.75), ('Custom', -1)],
    value=1.0, description='Speed:', style={'description_width': '60px'},
    layout=w.Layout(width='280px')
)
custom_speed = w.BoundedFloatText(
    value=1.0, min=0.1, max=10.0, step=0.1,
    description='Rate:', layout=w.Layout(width='200px', display='none'), style={'description_width': '40px'}
)
speed_sel.observe(make_toggle(custom_speed, -1), names='value')

display(w.VBox([
    w.HTML('<b>📦 Output format</b>'), format_sel,
    w.HTML('<hr style="margin:12px 0">'),
    settings_container,
    w.HTML('<hr style="margin:12px 0">'),
    w.HTML('<b>⚡ Playback speed</b>'), speed_sel, custom_speed,
]))

In [ ]:
# ⑤ Generate preview

if 'order_widgets' not in globals():
    raise RuntimeError('❌ order_widgets is not defined. Please run cell ③ first')

if order_widgets is None:
    ordered_videos = uploaded_videos
else:
    ordered_videos = [ow.value for ow in order_widgets]
    missing = [f for f in ordered_videos if f not in uploaded_videos]
    if missing:
        raise RuntimeError(
            f'❌ Selected file(s) not found: {missing}\n'
            '   Please re-run cell ③'
        )
    if len(set(ordered_videos)) != len(ordered_videos):
        dupes = [f for f in set(ordered_videos) if ordered_videos.count(f) > 1]
        raise ValueError(f'❌ Duplicate files detected: {dupes}')

print(f'📋 Merge order: {" → ".join(ordered_videos)}')

# Concatenate videos (if more than one)
if len(ordered_videos) == 1:
    source = ordered_videos[0]
else:
    try:
        with open('list.txt', 'w') as f:
            for v in ordered_videos:
                escaped = v.replace("\\", "\\\\").replace("'", "\\'")
                f.write(f"file '{escaped}'\n")
        _ffmpeg('-f', 'concat', '-safe', '0', '-i', 'list.txt', '-c', 'copy', 'combined.mov')
    finally:
        if os.path.exists('list.txt'):
            os.remove('list.txt')
    source = 'combined.mov'
    print(f'✅ Merged {len(ordered_videos)} files')

# Determine preview fps/scale from ④ settings (same resolution as final output)
if format_sel.value == 'gif':
    if gif_quality_sel.value == 'custom':
        prev_scale, prev_fps = gif_custom_scale.value, gif_custom_fps.value
    else:
        p = GIF_PRESETS[gif_quality_sel.value]
        prev_scale, prev_fps = p['scale'], p['fps']
else:
    prev_scale, prev_fps = mp4_scale.value, mp4_fps.value

speed = custom_speed.value if speed_sel.value == -1 else speed_sel.value
print(f'   Resolution: {prev_scale}px / fps: {prev_fps} / speed: {speed}x')
print('⏳ Generating preview...')

vf_parts = []
if speed != 1.0:
    vf_parts.append(f'setpts=PTS/{speed}')
vf_parts += [f'fps={prev_fps}', f'scale=trunc({prev_scale}/2)*2:-2:flags=lanczos']
vf_str = ','.join(vf_parts)

_ffmpeg('-i', source, '-vf', vf_str, '-c:v', 'libx264', '-crf', '23', '-preset', 'fast', '-an', 'preview.mp4')

size_mb = os.path.getsize('preview.mp4') / 1024 / 1024
print(f'\n✅ Done → preview.mp4 ({size_mb:.1f} MB)')
print('   ▶ ⑥ review preview → ⑦ zoom settings (optional) → ⑧ final export')


In [ ]:
# ⑥ Review preview
display(Video('preview.mp4', embed=True, width=720))

In [ ]:
# ⑦ Zoom settings (optional)
# Timeline: start → zoom in → hold → zoom out → back to normal

import base64
from google.colab import output as colab_output

AREA_GRID = {1:(0,0), 2:(1,0), 3:(2,0), 4:(0,1), 5:(1,1), 6:(2,1), 7:(0,2), 8:(1,2), 9:(2,2)}

zoom_enabled        = w.Checkbox(value=False, description='Add zoom', indent=False,
                                  layout=w.Layout(margin='4px 0', width='auto'))
zoom_events_box     = w.VBox([])
_zoom_getters       = []
_zoom_start_widgets = []

def make_area_grid():
    """Create a 3x3 ToggleButton grid; also returns get_area() that yields the selected cell number (1-9)."""
    LABELS = ['↖ Top-L', '↑ Top-C',  '↗ Top-R',
              '← Mid-L',  '● Center', '→ Mid-R',
              '↙ Bot-L',  '↓ Bot-C',  '↘ Bot-R']
    btns = [
        w.ToggleButton(
            description=LABELS[i], value=(i == 4),
            layout=w.Layout(width='88px', height='52px', margin='1px')
        ) for i in range(9)
    ]
    def on_click(change, clicked_idx):
        if change['new']:
            for j, b in enumerate(btns):
                if j != clicked_idx:
                    b.value = False
        elif not any(b.value for b in btns):
            btns[clicked_idx].value = True
    for idx, btn in enumerate(btns):
        btn.observe(lambda ch, i=idx: on_click(ch, i), names='value')
    grid = w.GridBox(btns, layout=w.Layout(
        grid_template_columns='repeat(3, 90px)', width='276px'
    ))
    get_area = lambda: next((i + 1 for i, b in enumerate(btns) if b.value), 5)
    return grid, get_area

def _lbl(text):
    """Right-aligned label."""
    return w.HTML(f'<div style="width:90px;text-align:right;padding-right:6px;line-height:32px">{text}</div>')

def _row(label, widget):
    return w.HBox([_lbl(label), widget])

def _get_video_duration():
    try:
        res = subprocess.run(
            ['ffprobe', '-v', 'error', '-show_entries', 'format=duration',
             '-of', 'default=noprint_wrappers=1:nokey=1', 'preview.mp4'],
            capture_output=True, text=True
        )
        return float(res.stdout.strip())
    except Exception:
        return None

def make_event_box(idx, default_start=0.0):
    grid, get_area = make_area_grid()
    max_z = w.BoundedFloatText(value=1.5, min=1.1, max=5.0, step=0.1,
        layout=w.Layout(width='120px'), tooltip='Peak zoom multiplier (1.1–5.0)')
    start = w.BoundedFloatText(value=default_start, min=0, max=3600,
        layout=w.Layout(width='120px'), tooltip='Timestamp in the video where zoom begins')
    in_d  = w.BoundedFloatText(value=0.2, min=0, max=60,
        layout=w.Layout(width='120px'), tooltip='Duration to ramp from 1x to peak zoom')
    hold  = w.BoundedFloatText(value=0.5, min=0, max=60,
        layout=w.Layout(width='120px'), tooltip='Duration to hold at peak zoom')
    out_d = w.BoundedFloatText(value=0.2, min=0, max=60,
        layout=w.Layout(width='120px'), tooltip='Duration to ramp from peak zoom back to 1x')

    vbox = w.VBox([
        w.HTML(f'<b>── Zoom Event #{idx} ──</b>'),
        w.HBox([
            w.VBox([w.HTML('<small>📍 Zoom area</small>'), grid]),
            w.VBox([
                _row('Max zoom:', max_z),
                w.HBox([_lbl('Start (s):'), start, w.HTML('<span style="font-size:12px;color:#666;margin-left:8px;line-height:32px">📍 update via button</span>')]),
                _row('In (s):', in_d),
                _row('Hold (s):', hold),
                _row('Out (s):', out_d),
            ], layout=w.Layout(margin='0 0 0 16px')),
        ]),
    ], layout=w.Layout(border='1px solid #ddd', padding='8px', margin='4px 0'))

    def get_params():
        return {
            'area':     get_area(),
            'max_z':    max_z.value,
            'start':    start.value,
            'in_dur':   in_d.value,
            'hold_dur': hold.value,
            'out_dur':  out_d.value,
        }
    return vbox, get_area, get_params, start

def _add_event(_):
    default_start = 0.0
    if _zoom_getters:
        p = _zoom_getters[-1]()
        end = p['start'] + p['in_dur'] + p['hold_dur'] + p['out_dur']
        default_start = round(end + 0.1, 2)
        dur = _get_video_duration()
        if dur is not None:
            default_start = min(default_start, max(0.0, dur - 0.1))
    vbox, _, get_params, start_widget = make_event_box(len(zoom_events_box.children) + 1, default_start)
    _zoom_getters.append(get_params)
    _zoom_start_widgets.append(start_widget)
    zoom_events_box.children = list(zoom_events_box.children) + [vbox]
    _update_record_target_options()

def _del_event(_):
    if zoom_events_box.children:
        _zoom_getters.pop()
        _zoom_start_widgets.pop()
        zoom_events_box.children = list(zoom_events_box.children)[:-1]
        _update_record_target_options()

add_btn = w.Button(description='+ Add Zoom Event', button_style='info',    layout=w.Layout(width='160px'))
add_btn.on_click(_add_event)
del_btn = w.Button(description='- Remove last',    button_style='warning', layout=w.Layout(width='140px'))
del_btn.on_click(_del_event)

record_target_sel = w.Dropdown(
    options=[], description='Target event:', style={'description_width': '100px'},
    layout=w.Layout(width='260px')
)

def _update_record_target_options():
    n = len(_zoom_start_widgets)
    record_target_sel.options = [(f'Event #{i+1}', i) for i in range(n)]
    if n > 0:
        record_target_sel.value = n - 1

def _record_position(time):
    """Called from JS: sets the current video position as the start time of the selected event."""
    if _zoom_start_widgets and record_target_sel.value is not None:
        _zoom_start_widgets[record_target_sel.value].value = round(float(time), 2)

colab_output.register_callback('record_position', _record_position)

if os.path.exists('preview.mp4'):
    with open('preview.mp4', 'rb') as _f:
        _b64 = base64.b64encode(_f.read()).decode()
    video_widget = w.HTML(f"""
<div style="margin:8px 0">
  <b>📺 Preview video</b><br>
  <video id="zoom_preview_video" width="720" controls style="display:block;margin:6px 0 4px">
    <source src="data:video/mp4;base64,{_b64}" type="video/mp4">
  </video>
  <button style="padding:4px 12px;cursor:pointer"
    onclick="(function(){{var v=document.getElementById('zoom_preview_video');google.colab.kernel.invokeFunction('record_position',[v.currentTime],{{}});}}())">
    📍 Record this position as start time
  </button>
</div>
""")
else:
    video_widget = w.HTML('<small>⚠️ preview.mp4 not found. Please run ⑤ first</small>')

record_row = w.HBox(
    [record_target_sel, add_btn, del_btn],
    layout=w.Layout(margin='2px 0 6px 0')
)

zoom_panel = w.VBox([
    video_widget,
    record_row,
    zoom_events_box,
    w.HTML(
        '<br><small>'
        '💡 How to use: play the preview to find timestamps → add Zoom Events and configure each → run ⑧<br>'
        '💡 Use the "📍 Record this position" button + target event dropdown to set any event\'s start time to the current playback position<br>'
        'Uncheck "Add zoom" to disable all zoom effects'
        '</small>'
    ),
], layout=w.Layout(display='none'))

def _on_zoom_toggle(change):
    if change['new'] and not zoom_events_box.children:
        _add_event(None)
    zoom_panel.layout.display = '' if change['new'] else 'none'
zoom_enabled.observe(_on_zoom_toggle, names='value')

display(w.VBox([
    w.HTML('<b>🔍 Zoom settings (skip this cell if not needed)</b>'),
    zoom_enabled,
    zoom_panel,
]))


In [ ]:
# ⑧ Final export
# Re-run only this cell after changing zoom settings

if 'source' not in globals():
    raise RuntimeError(
        '❌ source variable is not defined.\n'
        '   Please run cell ⑤ (Generate preview) first, then re-run this cell.'
    )

if not os.path.exists('preview.mp4'):
    raise RuntimeError('❌ preview.mp4 not found. Please run ⑤ first')

# Guard against stale previews when settings have changed
if format_sel.value == 'gif':
    expected_scale = (gif_custom_scale.value if gif_quality_sel.value == 'custom'
                      else GIF_PRESETS[gif_quality_sel.value]['scale'])
else:
    expected_scale = mp4_scale.value

_probe = subprocess.run(
    ['ffprobe', '-v', 'error', '-select_streams', 'v:0',
     '-show_entries', 'stream=width', '-of', 'csv=p=0', 'preview.mp4'],
    capture_output=True, text=True
)
_lines = _probe.stdout.strip().splitlines()
if not _lines or not _lines[0].isdigit():
    raise RuntimeError(f'❌ Failed to read preview.mp4 dimensions: {_probe.stderr}')
_out = _lines[0]
if int(_out) != expected_scale:
    raise RuntimeError(
        f'❌ Settings have changed (preview: {int(_out)}px / current setting: {expected_scale}px). '
        'Please re-run cell ⑤'
    )

out_format  = format_sel.value
OUTPUT_NAME = f'demo.{out_format}'

def _build_zoompan(events, pw, ph, fps):
    """Convert zoom events to an ffmpeg zoompan filter string.
    fps is specified explicitly to prevent speed drift.
    z_expr is inlined directly into x=/y= expressions because zoompan's z variable
    cannot be referenced inside those expressions.
    events must be sorted by start time ascending (guaranteed by caller).
    """
    z_expr, x_expr, y_expr = '1', '0', '0'
    for ev in reversed(events):
        col, row = AREA_GRID[ev['area']]
        cx  = pw * (2 * col + 1) / 6
        cy  = ph * (2 * row + 1) / 6
        s, i, h, o, mz = ev['start'], ev['in_dur'], ev['hold_dur'], ev['out_dur'], ev['max_z']
        sf  = s * fps
        inf = (s + i) * fps
        hf  = (s + i + h) * fps
        of  = (s + i + h + o) * fps

        denom_in  = max(inf - sf, 1e-4)
        denom_out = max(of - hf, 1e-4)
        z_in  = f'(1+({mz}-1)*(on-{sf:.4f})/{denom_in:.4f})'   if i > 0 else str(mz)
        z_out = f'({mz}-({mz}-1)*(on-{hf:.4f})/{denom_out:.4f})' if o > 0 else '1'
        ev_z  = (f'if(lt(on,{inf:.2f}),{z_in},'
                 f'if(lt(on,{hf:.2f}),{mz},'
                 f'if(lt(on,{of:.2f}),{z_out},{z_expr})))')
        z_expr = f'if(gte(on,{sf:.2f}),{ev_z},{z_expr})'

        zv     = f'({z_expr})'
        ev_x   = f'floor(max(0,min(iw-iw/{zv},{cx:.2f}-iw/(2*{zv}))))'
        ev_y   = f'floor(max(0,min(ih-ih/{zv},{cy:.2f}-ih/(2*{zv}))))'
        x_expr = f'if(between(on,{sf:.2f},{of:.2f}),{ev_x},{x_expr})'
        y_expr = f'if(between(on,{sf:.2f},{of:.2f}),{ev_y},{y_expr})'

    return f"zoompan=z='{z_expr}':x='{x_expr}':y='{y_expr}':d=1:s={pw}x{ph}:fps={fps}"

# ── Build zoom filter ────────────────────────────────────
zoom_filter = ''
if zoom_enabled.value and zoom_events_box.children:
    res = subprocess.run(
        ['ffprobe', '-v', 'error', '-select_streams', 'v:0',
         '-show_entries', 'stream=height', '-of', 'csv=p=0', 'preview.mp4'],
        capture_output=True, text=True
    )
    pw, ph = prev_scale, int(res.stdout.strip())
    detected_fps = prev_fps
    events = [getter() for getter in _zoom_getters]
    _sorted = sorted(events, key=lambda e: e['start'])
    for _i in range(len(_sorted) - 1):
        _e1, _e2 = _sorted[_i], _sorted[_i + 1]
        _e1_end = _e1['start'] + _e1['in_dur'] + _e1['hold_dur'] + _e1['out_dur']
        if _e2['start'] < _e1_end:
            raise ValueError(
                f"❌ Zoom events overlap: "
                f"event starting at {_e1['start']}s ends at {_e1_end:.2f}s, "
                f"which overlaps with event starting at {_e2['start']}s. "
                "Please adjust start times or durations."
            )
    zoom_filter = _build_zoompan(_sorted, pw, ph, detected_fps)
    print(f'🔍 Zoom: {len(events)} event(s) ({pw}x{ph} @ {detected_fps:.2f}fps)')

print(f'⏳ Exporting ({out_format.upper()})...')

# ── GIF output ──────────────────────────────────────────
if out_format == 'gif':
    if gif_quality_sel.value == 'custom':
        fps, colors, dither = gif_custom_fps.value, gif_custom_colors.value, gif_custom_dither.value
    else:
        p = GIF_PRESETS[gif_quality_sel.value]
        fps, colors, dither = p['fps'], p['colors'], p['dither']
    # When zoom is active, apply it to an intermediate MP4, then convert that to GIF
    if zoom_filter:
        _ffmpeg('-i', 'preview.mp4', '-vf', zoom_filter,
                '-c:v', 'libx264', '-crf', _ZOOM_INTERMEDIATE_CRF, '-preset', 'fast', '-an',
                'zoomed.mp4')
        gif_source = 'zoomed.mp4'
    else:
        gif_source = 'preview.mp4'
    filter_str = (
        f"[0:v] fps={fps},split [a][b];"
        f"[a] palettegen=max_colors={colors}:stats_mode=full [p];"
        f"[b][p] paletteuse=dither={dither}:diff_mode=rectangle"
    )
    _ffmpeg('-i', gif_source, '-filter_complex', filter_str, OUTPUT_NAME)

# ── MP4 output ──────────────────────────────────────────
elif out_format == 'mp4':
    crf = mp4_custom_crf.value if mp4_quality_sel.value == -1 else mp4_quality_sel.value
    _MP4_ENCODE_ARGS = ['-c:v', 'libx264', '-crf', crf, '-preset', 'slow', '-pix_fmt', 'yuv420p', '-an']

    if zoom_filter:
        # Apply zoom to preview.mp4 (already muted and speed-adjusted)
        _ffmpeg('-i', 'preview.mp4', '-vf', zoom_filter,
                *_MP4_ENCODE_ARGS, OUTPUT_NAME)
    else:
        _ffmpeg('-i', 'preview.mp4',
                *_MP4_ENCODE_ARGS, OUTPUT_NAME)

if os.path.exists('zoomed.mp4'):
    os.remove('zoomed.mp4')

size_mb = os.path.getsize(OUTPUT_NAME) / 1024 / 1024
print(f'\n✅ Done → {OUTPUT_NAME} ({size_mb:.1f} MB)')

if out_format == 'gif' and size_mb > 15:
    print('\n⚠️  File exceeds 15 MB. Try the following in ④:')
    print('   → Switch quality to "SNS / lightweight"')
    print('   → Or switch output format to MP4 for a much smaller file')

print('\n▶ Final preview:')
if out_format == 'gif':
    display(Image(OUTPUT_NAME))
else:
    display(Video(OUTPUT_NAME, embed=True, width=720))


In [ ]:
# ⑨ Choose save destination
save_sel = w.RadioButtons(
    options=[
        ('Download locally',     'local'),
        ('Save to Google Drive', 'drive'),
        ('Both',                 'both'),
    ],
    value='local', description='Save to:', style={'description_width': '60px'}
)
_fmt = out_format if 'out_format' in globals() else 'mp4'
drive_path = w.Text(
    value=f'MyDrive/demo.{_fmt}',
    description='Drive path:',
    layout=w.Layout(width='380px', display='none'),
    style={'description_width': '80px'}
)

def on_save_change(change):
    drive_path.layout.display = '' if change['new'] in ('drive', 'both') else 'none'
save_sel.observe(on_save_change, names='value')

display(w.VBox([w.HTML('<b>💾 Save destination</b>'), save_sel, drive_path]))

In [ ]:
# ⑨ Save
method = save_sel.value

if method in ('local', 'both'):
    colab_files.download(OUTPUT_NAME)
    print('✅ Downloaded locally')

if method in ('drive', 'both'):
    if not os.path.ismount('/content/drive'):
        drive.mount('/content/drive')
    _drive_val = drive_path.value
    if not _drive_val.endswith(f'.{out_format}'):
        _drive_val = os.path.splitext(_drive_val)[0] + f'.{out_format}'
    dest = f'/content/drive/{_drive_val.lstrip("/")}'
    os.makedirs(os.path.dirname(dest), exist_ok=True)
    shutil.copy(OUTPUT_NAME, dest)
    print(f'✅ Saved to Drive → {_drive_val}')